In [ ]:
def aspp_separableConvolution(x,filters):
  c=SeparableConvolution2D(filters,(3,3),strides=1,padding="same",dilation_rate=6)(x)
  c=Activation("relu")(c)
  c=SeparableConvolution2D(filters,(1,1),strides=1,padding="same")(c)
  c=Activation("relu")(c)
  c=SeparableConvolution2D(filters,(1,1),strides=1,padding="same")(c)
  c=Activation("relu")(c)

  a=SeparableConvolution2D(filters,(3,3),strides=1,padding="same",dilation_rate=12)(x)
  a=Activation("relu")(a)
  a=SeparableConvolution2D(filters,(1,1),strides=1,padding="same")(a)
  a=Activation("relu")(a)
  a=SeparableConvolution2D(filters,(1,1),strides=1,padding="same")(a)
  a=Activation("relu")(a)

  b=SeparableConvolution2D(filters,(3,3),strides=1,padding="same",dilation_rate=18)(x)
  b=Activation("relu")(b)
  b=SeparableConvolution2D(filters,(1,1),strides=1,padding="same")(b)
  b=Activation("relu")(b)
  b=SeparableConvolution2D(filters,(1,1),strides=1,padding="same")(b)
  b=Activation("relu")(b)

  d=SeparableConvolution2D(filters,(3,3),strides=1,padding="same",dilation_rate=24)(x)
  d=Activation("relu")(d)
  d=SeparableConvolution2D(filters,(1,1),strides=1,padding="same")(d)
  d=Activation("relu")(d)
  d=SeparableConvolution2D(filters,(1,1),strides=1,padding="same")(d)
  d=Activation("relu")(d)

  out=Concatenate()([c,a,b,d])

  out=SeparableConvolution2D(256,(1,1),strides=1,padding="same")(out)
  out=Activation("relu")(out)

  return out


In [ ]:
def SCPPnet():
  image_shape = (512,512,3)
  inputs= Input(shape=image_shape)
  x=inputs
  #Encoder
  x1=Convolution2D(32,(3,3),strides=1,padding="same")(x)
  x1=Activation("relu")(x1)
  x1=Convolution2D(32,(3,3),strides=1,padding="same")(x1)
  x1=Activation("relu")(x1)
  
  p1=MaxPooling2D(pool_size=(2,2))(x1)
  #print(p1)
  x2=Convolution2D(64,(3,3),strides=1,padding="same")(p1)
  x2=Activation("relu")(x2)
  x2=Convolution2D(64,(3,3),strides=1,padding="same")(x2)
  x2=Activation("relu")(x2)
  
  p2=MaxPooling2D(pool_size=(2,2))(x2)
  #print(p2)
  x3=Convolution2D(128,(3,3),strides=1,padding="same")(p2)
  x3=Activation("relu")(x3)
  x3=Convolution2D(128,(3,3),strides=1,padding="same")(x3)
  x3=Activation("relu")(x3)
  x3=Convolution2D(128,(3,3),strides=1,padding="same")(x3)
  x3=Activation("relu")(x3)
  
  p3=MaxPooling2D(pool_size=(2,2))(x3)
  #print(p3)
  x4=Convolution2D(256,(3,3),strides=1,padding="same")(p3)
  x4=Activation("relu")(x4)
  x4=Convolution2D(256,(3,3),strides=1,padding="same")(x4)
  x4=Activation("relu")(x4)
  x4=Convolution2D(256,(3,3),strides=1,padding="same")(x4)
  x4=Activation("relu")(x4)
  p4=MaxPooling2D(pool_size=(2,2))(x4)

  #aspp
  p4=aspp_separableConvolution(p4,512)
  
  



  u1=UpSampling2D(size=(16,16))(p4)
  #print(u1)
  u1=Convolution2D(2,(3,3),strides=1,padding="same")(u1)
  u1=Activation("relu")(u1)
  u1=Convolution2D(2,(1,1),strides=1,padding="same")(u1)
  u1=Activation("relu")(u1)
  
  
  
  u2=UpSampling2D(size=(8,8))(x4)
  #print(u2)
  u2=Convolution2D(2,(3,3),strides=1,padding="same")(u2)
  u2=Activation("relu")(u2)
  u2=Convolution2D(2,(1,1),strides=1,padding="same")(u2)
  u2=Activation("relu")(u2)

  
 
 

  u3=UpSampling2D(size=(4,4))(x3)
  #print(u3)
  u3=Convolution2D(2,(3,3),strides=1,padding="same")(u3)
  u3=Activation("relu")(u3)
  u3=Convolution2D(2,(1,1),strides=1,padding="same")(u3)
  u3=Activation("relu")(u3)

  
  
  
  
  output = Add()([u1,u2,u3])
 


  output=Convolution2D(1,(1,1),strides=1,padding="same")(output)
  output=Activation("sigmoid")(output)
  model = Model(inputs,output)

  return model


In [ ]:
model_SCPPnet = SCPPnet()
adam = optimizers.Adam(lr=0.0001, decay=5*1e-7)
model_SCPPnet.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])
model_SCPPnet.summary()